<a href="https://colab.research.google.com/github/ankita2002/LLMS/blob/main/Intro2LLM_Homework_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 02: Retrieval with TF-IDF / BM25 / LSA, Word2Vec (CBOW), and MAP Evaluation

### This homework has three tasks:
1) **Ranked retrieval with TF-IDF, BM25, and LSA (Truncated SVD)** on a sampled subset of 20 Newsgroups — **6 pts**
2) **Word2Vec (CBOW) with PyTorch** trained on the same sampled subset — **7 pts**
3) **Mean Average Precision (MAP)** implementation + evaluate LSA vs CBOW on the test split — **7 pts**

**BONUS:** Solve any task with an LLM — **2 pts**

#### Total: 20 points (+2 bonus)

## Pre-requisite code

The code in this section will be used in the tasks.
**Do not change these lines in your submission.**

We:
- load a **small, fixed subset** of the 20 Newsgroups dataset (4 topics),
- split it into TRAIN_DOCS and TEST_DOCS,
- build queries and relevance sets,
- define helpers for tokenization, cosine similarity, etc.

In [1]:
# Run these on Colab.
!pip -q install rank_bm25 torch scikit-learn

In [2]:
import math, random, collections, itertools
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import numpy as np

# External libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.datasets import fetch_20newsgroups
from rank_bm25 import BM25Okapi

# Reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# -------------------------------------------------
# Tokenization
# -------------------------------------------------
def tokenize(text: str) -> List[str]:
    """
    Lowercase, keep alphabetic tokens only, split on whitespace.
    We'll also use this tokenizer inside scikit-learn's TfidfVectorizer.
    """
    toks = []
    for t in text.lower().strip().split():
        # keep only alphabetic chars
        t = "".join(ch for ch in t if ch.isalpha())
        if t:
            toks.append(t)
    return toks

@dataclass
class Doc:
    text: str
    tokens: List[str]
    label: str      # coarse topic label (used to derive relevance)

def make_docs(lines: List[Tuple[str, str]]) -> List[Doc]:
    """
    lines = [(text, label), ...]
    We'll re-tokenize here, even though we already truncated earlier,
    to keep the interface clean.
    """
    docs = []
    for txt, lab in lines:
        docs.append(Doc(text=txt, tokens=tokenize(txt), label=lab))
    return docs

# -------------------------------------------------
# Mini 20 Newsgroups subset
# -------------------------------------------------
# We'll sample 4 topical newsgroups as our "topics"
CATEGORIES = [
    "comp.sys.mac.hardware",  # Apple / Mac hardware issues
    "sci.space",              # Space / NASA / missions
    "rec.autos",              # Cars / engines / performance
    "rec.sport.hockey",       # Hockey / teams / playoffs
]

# We'll map each full newsgroup name to a short label
LABEL_MAP = {
    "comp.sys.mac.hardware": "mac",
    "sci.space": "space",
    "rec.autos": "autos",
    "rec.sport.hockey": "hockey",
}

# How many docs we keep per class for train/test,
# and how aggressively we truncate each post.
TRAIN_PER_CLASS = 100
TEST_PER_CLASS  = 5
MAX_TOKENS_PER_DOC = 50  # cap each doc to first N tokens for speed

def collect_subset(dataset, per_class: int) -> List[Tuple[str,str]]:
    """
    Build a list of (truncated_text, short_label) for up to per_class
    examples per category in CATEGORIES.
    We remove headers/footers/quotes via fetch_20newsgroups(... remove=...).
    """
    by_class: Dict[str, List[Tuple[str,str]]] = {cat: [] for cat in CATEGORIES}

    for text, tgt in zip(dataset.data, dataset.target):
        cat = dataset.target_names[tgt]
        if cat not in by_class:
            continue
        if len(by_class[cat]) >= per_class:
            continue

        toks = tokenize(text)
        toks = toks[:MAX_TOKENS_PER_DOC]        # truncate
        truncated_text = " ".join(toks)         # rejoin to text form

        by_class[cat].append((truncated_text, LABEL_MAP[cat]))

    # flatten in fixed category order for determinism
    lines = []
    for cat in CATEGORIES:
        lines.extend(by_class[cat])
    return lines

# Load raw 20NG subsets (without headers/footers/quotes to reduce noise)
_20ng_train = fetch_20newsgroups(
    subset="train",
    categories=CATEGORIES,
    remove=("headers", "footers", "quotes"),
    shuffle=False,
)

_20ng_test = fetch_20newsgroups(
    subset="test",
    categories=CATEGORIES,
    remove=("headers", "footers", "quotes"),
    shuffle=False,
)

# Build our small train/test corpora
TRAIN_RAW = collect_subset(_20ng_train, per_class=TRAIN_PER_CLASS)
TEST_RAW  = collect_subset(_20ng_test,  per_class=TEST_PER_CLASS)

TRAIN_DOCS = make_docs(TRAIN_RAW)
TEST_DOCS  = make_docs(TEST_RAW)

# -------------------------------------------------
# Queries & relevance
# -------------------------------------------------
# We craft one realistic search-style query per topic.
# Relevant docs are those in TEST_DOCS whose label matches.
QUERIES = [
    (
        "how do i fix problems on my mac like disk failures, monitor issues and software failures",
        "mac"
    ),
    (
        "latest updates on nasa missions space shuttle launches and orbital science experiments",
        "space"
    ),
    (
        "advice on car performance engine reliability and buying a new vehicle",
        "autos"
    ),
    (
        "discussion about hockey teams playoffs goalies and physical play on the ice",
        "hockey"
    )
]

def relevant_doc_indices(test_docs: List[Doc], label: str) -> List[int]:
    """Return indices in test_docs whose label matches the given topic label."""
    return [i for i, d in enumerate(test_docs) if d.label == label]

print(f"✓ Loaded {len(TRAIN_DOCS)} train docs and {len(TEST_DOCS)} test docs")
for lab in sorted({d.label for d in TRAIN_DOCS}):
    print("  – label:", lab,
          "| train:", sum(1 for d in TRAIN_DOCS if d.label==lab),
          "| test:",  sum(1 for d in TEST_DOCS  if d.label==lab))

# -------------------------------------------------
# Vector helpers
# -------------------------------------------------
def cosine(a: np.ndarray, b: np.ndarray) -> float:
    """Cosine similarity between vectors a and b. Returns 0 if either vector has zero norm."""
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return float(np.dot(a, b) / (norm_a * norm_b))

# -------------------------------------------------
# Vocabulary utilities (for Task 2 CBOW)
# -------------------------------------------------
def build_vocab(docs: List[Doc], min_count: int = 1
               ) -> Tuple[Dict[str,int], Dict[int,str], Dict[str,int]]:
    """
    Build word-index mappings based on frequency in a list of Doc objects.

    Returns:
        w2i: word -> index
        i2w: index -> word
        freq: Counter of word frequencies
    """
    freq = collections.Counter(t for d in docs for t in d.tokens)
    vocab = [w for w, c in freq.items() if c >= min_count]
    vocab = sorted(vocab)
    w2i = {w:i for i, w in enumerate(vocab)}
    i2w = {i:w for w, i in w2i.items()}
    return w2i, i2w, freq

print("✓ Pre-req code ready.")

✓ Loaded 400 train docs and 20 test docs
  – label: autos | train: 100 | test: 5
  – label: hockey | train: 100 | test: 5
  – label: mac | train: 100 | test: 5
  – label: space | train: 100 | test: 5
✓ Pre-req code ready.


# TASK 1 (6 pts): Ranked Retrieval with TF-IDF, BM25, and LSA

**Goal.** We’ll build *three* retrieval models on a small subset of the
20 Newsgroups dataset and compare how they rank documents:

1. **TF-IDF ranked retrieval** - use scikit-learn’s `TfidfVectorizer`
2. **BM25 (Okapi BM25)** - use the rank_bm25 library’s `BM25Okapi`
3. **LSA (Latent Semantic Analysis via Truncated SVD)** - use scikit-learn’s `TruncatedSVD` on the same TF-IDF features

We'll then run the queries in `QUERIES` and see how each method ranks
the held-out TEST_DOCS. We’ll also:
- check vector dimensionality (TF-IDF / BM25 vocab size vs. LSA k),
- compute **Precision@5 (P@5)** for each method (per query and averaged).

**Retrieval setup**
- Candidate results are the **TEST_DOCS** we built above (5 per topic).
- For each query `(query_text, label)`:
  - Rank all TEST_DOCS using TF-IDF.
  - Rank all TEST_DOCS using BM25.
  - Rank all TEST_DOCS using LSA.
  - Show the top-5 doc labels.
  - Report P@5.

**What you implement**

**TF-IDF ranked retrieval (2 pts)**  
- Fit a `TfidfVectorizer` on the retrieval corpus (TEST_DOCS).
- Use our provided tokenizer;
- Implement `rank_with_tfidf(query_text, vectorizer, doc_matrix)`:
  - Transform the query with the same `TfidfVectorizer`.
  - Score each document by aggregating TF-IDF weights over the query terms.
  - Return doc IDs sorted by score (descending).

**BM25 retrieval (2 pts)**  
- Fit a `BM25Okapi` model on the tokenized docs.
- Implement `rank_with_bm25(query_text, bm25_model)`:
  - Tokenize the query with our tokenizer.
  - Score all docs via BM25. Use `get_scores(query_tokens)` from `BM25Okapi` to get aggregated BM25 scores.
  - Return doc IDs sorted by score (descending).

**LSA retrieval (2 pts)**
- Reuse the same TF-IDF vectorizer and its TF-IDF matrix used for TF-IDF ranking, then apply `TruncatedSVD` on that TF-IDF matrix to build a low-dimensional semantic space (LSA).
- Implement `rank_with_lsa(query_text, lsa_model)`:
  - Project query into that same semantic space (via the same TF-IDF vectorizer + SVD).
  - Rank by cosine similarity in LSA space.

**Grading (6 pts total)**
- **1.1** TF-IDF retrieval implementation — **2 pts**  
- **1.2** BM25 retrieval implementation — **2 pts**  
- **1.3** LSA retrieval implementation — **2 pts**


In [ ]:
# -----------------
# TF-IDF ranked retrieval (scikit-learn)
# -----------------
def fit_tfidf_ranker(docs: List[Doc]) -> Tuple[TfidfVectorizer, np.ndarray]:
    """
    Fit a TfidfVectorizer on the given docs (retrieval corpus).
    Returns:
        vectorizer: fitted TfidfVectorizer
        doc_matrix: TF-IDF matrix of shape (num_docs, vocab_size)
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


def rank_with_tfidf(query_text: str,
                    vectorizer: TfidfVectorizer,
                    doc_matrix) -> List[int]:
    """
    Rank docs using aggregated TF-IDF weights.
    You can use the dot product between the query vector and TF-IDF matrix.
    Returns:
        order: list of doc indices sorted by score desc
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


# -----------------
# BM25 ranked retrieval (rank_bm25)
# -----------------
def fit_bm25_ranker(docs: List[Doc]) -> BM25Okapi:
    """
    Fit BM25 on the given docs (list of token lists).
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


def rank_with_bm25(query_text: str,
                   bm25_model: BM25Okapi) -> List[int]:
    """
    Rank docs by aggregated BM25 score.
    Returns:
        order: doc indices (0..num_docs-1) sorted by score desc
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


# -----------------
# LSA ranked retrieval (reuse TF-IDF + TruncatedSVD)
# -----------------
@dataclass
class LSAModel:
    """
    LSAModel stores:
    - vectorizer: the same fitted TfidfVectorizer used for TF-IDF ranking
    - svd: fitted TruncatedSVD
    - doc_embeddings: dense array of shape (num_docs, k)
    - k: number of latent dimensions used
    """
    vectorizer: TfidfVectorizer
    svd: TruncatedSVD
    doc_embeddings: np.ndarray
    k: int


def fit_lsa_ranker(vectorizer: TfidfVectorizer,
                   doc_matrix,  # TF-IDF matrix constructed before
                   k: int=10,
                   random_state: int=RANDOM_SEED) -> LSAModel:
    """
    Steps:
    1) Reuse the same TF-IDF matrix constructed before
    2) Reduce dimensionality with TruncatedSVD
    3) Store doc embeddings in latent semantic space
    """
    # clamping k so it doesn't exceed the TF-IDF feature dimension
    max_k = max(1, min(k, min(doc_matrix.shape) - 1))
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE
    return LSAModel(
        vectorizer=vectorizer,
        svd=svd,
        doc_embeddings=doc_embeddings,
        k=max_k
    )


def rank_with_lsa(query_text: str, lsa_model: LSAModel) -> List[int]:
    """
    Rank docs by cosine similarity in the LSA semantic space.
    (Query -> TF-IDF -> SVD; then cosine vs doc embeddings.)
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


# -------------------------------------------------
# Build all three rankers on the same retrieval corpus (TEST_DOCS)
# -------------------------------------------------
tfidf_vec, tfidf_matrix = fit_tfidf_ranker(TEST_DOCS)
bm25_model             = fit_bm25_ranker(TEST_DOCS)
lsa_model              = fit_lsa_ranker(tfidf_vec, tfidf_matrix, k=10, random_state=RANDOM_SEED)

# Vector size comparison:
tfidf_vocab_size = len(tfidf_vec.vocabulary_)   # dimensionality of TF-IDF term space
bm25_vocab_size  = len(bm25_model.idf)          # dimensionality of BM25 term space
lsa_k            = lsa_model.k                  # dimensionality of LSA latent semantic space

print("\n[TASK1] Vector Size Comparison:")
print(f"  TF-IDF dim   = {tfidf_vocab_size}")
print(f"  BM25 dim     = {bm25_vocab_size}")
print(f"  LSA dim (k)  = {lsa_k}")


# -------------------------------------------------
# Evaluation: Precision@5 (per query and averaged)
# -------------------------------------------------
def precision_at_k(ranked: List[int], relevant: set, k: int = 5) -> float:
    hits = sum(i in relevant for i in ranked[:k])
    return hits / float(k)

TOPK = 5
tfidf_p5_scores = []
bm25_p5_scores  = []
lsa_p5_scores   = []

for qtext, qlab in QUERIES:
    rel = set(relevant_doc_indices(TEST_DOCS, qlab))

    tfidf_order = rank_with_tfidf(qtext, tfidf_vec, tfidf_matrix)
    bm25_order  = rank_with_bm25(qtext, bm25_model)
    lsa_order   = rank_with_lsa(qtext, lsa_model)

    p5_tfidf = precision_at_k(tfidf_order, rel, TOPK)
    p5_bm25  = precision_at_k(bm25_order,  rel, TOPK)
    p5_lsa   = precision_at_k(lsa_order,   rel, TOPK)

    tfidf_p5_scores.append(p5_tfidf)
    bm25_p5_scores.append(p5_bm25)
    lsa_p5_scores.append(p5_lsa)

    print(f"\n[TASK1] Query='{qtext}' | (label={qlab})")
    print("   TF-IDF top-5 labels:", [TEST_DOCS[i].label for i in tfidf_order[:TOPK]],
          f"| P@5={p5_tfidf:.3f}")
    print("   BM25  top-5 labels:", [TEST_DOCS[i].label for i in bm25_order[:TOPK]],
          f"| P@5={p5_bm25:.3f}")
    print("   LSA   top-5 labels:", [TEST_DOCS[i].label for i in lsa_order[:TOPK]],
          f"| P@5={p5_lsa:.3f}")

print("\n[TASK1 RESULT_CHECKING_POINT] Precision@5 (average across queries):")
print(f"  TF-IDF: ({np.mean(tfidf_p5_scores):.3f})")
print(f"  BM25:   ({np.mean(bm25_p5_scores):.3f})")
print(f"  LSA:    ({np.mean(lsa_p5_scores):.3f})")


In [ ]:
# ------------------------- TESTS & EXPECTED OUTPUT -------------------------
# [TASK1] Vector Size Comparison:
#   TF-IDF dim   = 441
#   BM25 dim     = 441
#   LSA dim (k)  = 10
#
# [TASK1] Query='how do i fix problems on my mac ...' | (label=mac)
#    TF-IDF top-5 document labels: ['space', 'autos', ...] | P@5=0.400
#    BM25  top-5 document labels: ['space', 'mac', ...] | P@5=0.400
#    LSA   top-5 document labels: ['space', 'mac', ...] | P@5=0.400
#
# ...
#
# [TASK1] Query='discussion about hockey teams ...' | (label=hockey)
#    TF-IDF top-5 document labels: ['hockey', 'space', ...] | P@5=0.400
#    BM25  top-5 document labels: ['hockey', 'space', ...] | P@5=0.400
#    LSA   top-5 document labels: ['hockey', 'space', ...] | P@5=0.400
#
# [TASK1 RESULT_CHECKING_POINT] Precision@5 (average across queries):
#   TF-IDF: (0.400)
#   BM25:   (0.450)
#   LSA:    (0.400)
# ---------------------------------------------------------------------------

# TASK 2 (7 pts): Word2Vec (CBOW) with PyTorch

## Goal
Train a **CBOW** model on the **train subset of 20 Newsgroups** (**exactly 100 docs per topic, truncated**) using **PyTorch** and NumPy utilities.

**Do not** use external libraries that already implement CBOW (e.g., *gensim*).  

- **Input:** mean of embeddings of the `2W` context tokens around a center word  
- **Output:** logits over the whole vocabulary predicting the **center** word  
- **Loss:** cross-entropy (`nn.CrossEntropyLoss`)  
- **Optimization:** **Adam optimizer** with **mini-batches** (`DataLoader` is used to easily prepare and load mini-batches)

We use a **fixed window** of size `W` on each side; contexts are **exactly `2W` tokens**.  
Positions that don’t have enough neighbors (near boundaries) are **skipped** — no padding/masking needed.

We also use a **naïve softmax** output (complexity **$O(|V|)$** per update) — this is **okay** for our small setup.

### Parameter shapes
Let $|V|$ be the vocabulary size and $d$ the embedding dimension:
- $ W_{\text{in}} \in \mathbb{R}^{|V|\times d} $
- $ W_{\text{out}} \in \mathbb{R}^{|V|\times d} $
- $ b_{\text{out}} \in \mathbb{R}^{|V|} $

## What you implement
1. **Vocab & data**
   - Build `(word → index)` / `(index → word)` from the **train** texts (already implemented for you).
   - Create CBOW pairs `(context_indices, target_index)` using a symmetric window of size `W`.
   - **Skip** positions that don’t yield a full `2W`-token context (typically happens at edge positions or short sentences).

2. **Model (PyTorch)**
   - `nn.Embedding(|V|, d)` → **mean over the `2W` context embeddings** → `nn.Linear(d, |V|)` → logits.
   - Use `nn.CrossEntropyLoss()` (softmax is automatically calculated in this utility).

3. **Training (mini-batches + Adam)**
   - Wrap pairs in a `Dataset`/`DataLoader` (`batch_size = B`).
   - Train for `E` epochs with `torch.optim.Adam(model.parameters(), lr=lr)`.
   - Print average training loss per epoch.

4. **Embedding helper**
   - `embed_text_mean(tokens, cbow_model)`: average learned **input** embeddings to represent any text (it will be used in Task 3).

## Grading (7 pts)
- **2.1** Data maker (windowing, indexing) — **2 pts**
- **2.2** Training loop (mini-batches + Adam) — **3 pts**
- **2.3** Embedding helper (mean of tokens embeddings) — **2 pts**  


> **Notes**
> - We will compare CBOW vs LSA retrieval in **Task 3**.


In [ ]:
import numpy as np
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# -----------------------------
# Reproducibility
# -----------------------------
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Vocab utilities for CBOW, build ing (word -> index) and (index -> word)
# -----------------------------
def build_w2v_vocab(docs: List[Doc], min_count: int = 1
                   ) -> Tuple[Dict[str,int], Dict[int,str]]:
    """
    Build a vocabulary from TRAIN_DOCS.
    Returns:
        w2i: word -> index  (0..|V|-1)
        i2w: index -> word
    """
    w2i, i2w, freq = build_vocab(docs, min_count=min_count)
    return w2i, i2w

def cbow_training_data_fixed_window(docs: List[Doc],
                                    w2i: Dict[str,int],
                                    window: int = 2
                                   ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Create CBOW pairs with a fixed-size symmetric window of size `window`.
    For each token position c, we require exactly `2*window` context tokens.
    Positions that don't have enough neighbors are skipped.

    Returns:
        contexts: np.ndarray of shape (N, 2*window) with word indices
        targets : np.ndarray of shape (N,) with target word indices
    """
    contexts, targets = [], []
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE
    return np.array(contexts, dtype=np.int64), np.array(targets, dtype=np.int64)

# -----------------------------
# PyTorch Dataset Format
# -----------------------------
class CBOWDataset(Dataset):
    def __init__(self, contexts: np.ndarray, targets: np.ndarray):
        self.contexts = contexts
        self.targets  = targets
    def __len__(self) -> int:
        return self.targets.shape[0]
    def __getitem__(self, idx: int):
        x = torch.tensor(self.contexts[idx], dtype=torch.long)
        y = torch.tensor(self.targets[idx],  dtype=torch.long)
        return x, y

# -----------------------------
# CBOW Model (Embedding mean -> Linear)
# -----------------------------
class CBOWTorch(nn.Module):
    def __init__(self, vocab_size: int, dim: int):
        super().__init__()
        self.in_embed = nn.Embedding(vocab_size, dim) # (|V|, d)
        self.out = nn.Linear(dim, vocab_size)         # (d, |V|)
        # The following lines are optional but common in word embedding models:
        # Small std (0.01) keeps initial logits near zero so softmax isn't saturated.
        # This stabilizes early training (gradients stay useful and not too peaky).
        nn.init.normal_(self.in_embed.weight, mean=0.0, std=0.01)
        nn.init.normal_(self.out.weight,      mean=0.0, std=0.01)
        nn.init.zeros_(self.out.bias)

    def forward(self, ctx_idxs: torch.LongTensor) -> torch.Tensor:
        """
        ctx_idxs: (B, 2W) tensor of word indices
        Returns:
            logits: (B, |V|)
        """
        emb = self.in_embed(ctx_idxs)        # (B, 2W, d)
        h   = emb.mean(dim=1)                # (B, d)   mean over context tokens
        logits = self.out(h)                 # (B, |V|)
        return logits

@dataclass
class CBOWModelTorch:
    """Light wrapper to carry the trained model and basic metadata."""
    w2i: Dict[str,int]
    i2w: Dict[int,str]
    model: nn.Module
    dim: int
    window: int

# -----------------------------
# CBOW Model Training Function
# -----------------------------
def cbow_train_torch(train_docs: List[Doc],
                     dim: int        = 100,
                     window: int     = 2,
                     batch_size: int = 256,
                     epochs: int     = 10,
                     lr: float       = 1e-3,
                     seed: int       = RANDOM_SEED
                    ) -> CBOWModelTorch:
    """
    Train CBOW with fixed-size windows using Adam and mini-batches.
    """
    # Build vocab and pairs
    w2i, i2w = build_w2v_vocab(train_docs)
    contexts, targets = cbow_training_data_fixed_window(train_docs, w2i, window=window)
    print(f"[TASK2] CBOW data: {len(targets)} pairs | V={len(w2i)} | d={dim} | window={window}")

    ds = CBOWDataset(contexts, targets)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE
    return CBOWModelTorch(w2i=w2i, i2w=i2w, model=model, dim=dim, window=window)

# -----------------------------
# Embedding helper for downstream tasks
# -----------------------------
def embed_text_mean(tokens: List[str], cbow: CBOWModelTorch) -> np.ndarray:
    """
    Average input embeddings for provided tokens.
    Returns a numpy vector of shape (d,).
    """
    vec = np.zeros(cbow.dim, dtype=float)
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE
    return vec

# -----------------------------
# Train on TRAIN_DOCS (example settings)
# -----------------------------
cbow = cbow_train_torch(
    TRAIN_DOCS,
    dim=100,          # embedding dim
    window=2,         # context window on each side
    batch_size=256,
    epochs=10,
    lr=1e-3,
    seed=RANDOM_SEED
)

# Quick sanity: embed the first train doc
ex_vec = embed_text_mean(TRAIN_DOCS[0].tokens, cbow)
print("[TASK2 RESULT_CHECKING_POINT] Embedding shape:", ex_vec.shape)


In [ ]:
# ------------------------- TESTS & EXPECTED OUTPUT -------------------------
# [TASK2] CBOW data: 15152 pairs | V=4015 | d=100 | window=2
#   epoch 04/10 | avg_loss6.8..
#   ...
#   epoch 10/10 | avg_loss=6.5..
# [TASK2 RESULT_CHECKING_POINT] Embedding shape: (100,)
# ---------------------------------------------------------------------------

# TASK 3 (7 pts): MAP implementation — compare & evaluate LSA vs CBOW on the test set

**Reminder:** You must finish Task 1 (to get `lsa_model`) and Task 2 (to get `cbow`) before starting Task 3. We will use those two models here.

## Goal
1. Implement **Average Precision (AP)** and **Mean Average Precision (MAP)**.
2. Use them to evaluate and compare:
   - **LSA retrieval** (from Task 1, using TruncatedSVD on TF-IDF)
   - **CBOW retrieval** (from Task 2, using learned word embeddings)
3. Report how well each model retrieves relevant documents for each query.
4. Compare their **embedding dimensionality**:
   - LSA works in a low-rank semantic space of size `k`.
   - CBOW uses learned embeddings of size `d`.

## Retrieval setup
- We have `TEST_DOCS` and `QUERIES`.
- For each query `(query_text, label)`:
  - A test doc is **relevant** if `doc.label == label`.
  - We produce a ranked list of all test docs.

**LSA ranking (Task 1 model):**
1. Convert the query to TF-IDF with `lsa_model.vectorizer`.
2. Project it into the LSA space with `lsa_model.svd`.
3. Compute cosine similarity with each document embedding in `lsa_model.doc_embeddings`.
4. Sort by similarity (highest first).

**CBOW ranking (Task 2 model):**
1. Embed the query by averaging the CBOW input embeddings (`W_in`) of its tokens.
2. Embed each test doc the same way.
3. Rank by cosine similarity.

## Evaluation metrics
Let the ranked documents for a query be $ d_1, d_2, \dots, d_n $, and let $ R $ be the set of relevant doc indices.

**Precision@k** is:
$$
\mathrm{P@}k \;=\; \frac{TP@k}{TP@k + FP@k}.
$$

Define the **relevance indicator**
$$
\mathrm{rel}_k \;=\;
\begin{cases}
1 & \text{if } d_k \in R,\\
0 & \text{otherwise.}
\end{cases}
$$

**Average Precision (AP)** for one query is:
$$
\mathrm{AP}
\;=\;
\frac{1}{|R|}
\sum_{k=1}^{n}
\mathrm{P@}k \cdot \mathrm{rel}_k.
$$

**Mean Average Precision (MAP)** over all queries is:
$$
\mathrm{MAP} = \frac{1}{Q} \sum_{q=1}^{Q} \mathrm{AP}_q
$$

We will:
- compute AP for each query,
- average to get MAP for **LSA** and **CBOW**

### Grading (7 pts)
- **3.1** `average_precision` and `mean_average_precision` — **3 pts**  
- **3.2** LSA ranking with MAP — **2 pts**  
- **3.3** CBOW ranking with MAP — **2 pts**


In [ ]:
# -------------------------------------------------
# AP and MAP implementation
# -------------------------------------------------
def average_precision(ranking: List[int], relevant: set) -> float:
    """
    Compute Average Precision (AP) for a single query.
    ranking: list of doc indices in ranked order (best first).
    relevant: set of doc indices considered relevant for this query.

    AP = (1/|R|) * sum_k Precision@k * rel_k
    """
    if not ranking or not relevant:
        return 0.0
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


def mean_average_precision(all_rankings: List[List[int]],
                           all_relevant: List[set]) -> float:
    """
    Compute MAP across multiple queries.
    all_rankings[i] is the ranked doc indices for query i
    all_relevant[i] is the set of relevant doc indices for query i

    MAP = mean_i AP_i
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


# -------------------------------------------------
# Retrieval with LSA and CBOW for MAP evaluation
# -------------------------------------------------

# CBOW embeddings for all TEST_DOCS
CBOW_TEST_EMBEDS = [embed_text_mean(d.tokens, cbow) for d in TEST_DOCS]  # (num_docs, d)

# LSA embeddings for all TEST_DOCS were already computed in lsa_model.doc_embeddings
LSA_TEST_EMBEDS = lsa_model.doc_embeddings  # shape: (num_docs, lsa_model.k)


def rank_with_lsa_map(query_text: str,
                      lsa_model: LSAModel,
                      test_embeds: np.ndarray) -> List[int]:
    """
    Rank TEST_DOCS for a query using the LSA semantic space.
    1) TF-IDF the query with lsa_model.vectorizer
    2) Project to LSA with lsa_model.svd
    3) Cosine sim vs each doc embedding (test_embeds)
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


def rank_with_cbow_map(query_text: str,
                       cbow_model: CBOWModelTorch,
                       test_embeds: List[np.ndarray]) -> List[int]:
    """
    Rank TEST_DOCS for a query using CBOW embeddings.
    1) Average word vectors for the query text
    2) Cosine similarity vs each precomputed doc embedding
    3) Sort high -> low
    """
    ## YOUR_CODE_STARTS_HERE
    ## YOUR_CODE_ENDS_HERE


# -------------------------------------------------
# Dimensionality comparison
# -------------------------------------------------
lsa_dim  = lsa_model.k      # LSA latent dimension k
cbow_dim = cbow.dim         # CBOW embedding dimension d

print("[TASK3] Embedding dimensionality:")
print(f"  LSA  dim (k) = {lsa_dim}")
print(f"  CBOW dim (d) = {cbow_dim}")
print()

# -------------------------------------------------
# MAP evaluation + reporting
# -------------------------------------------------
lsa_rankings  = []
cbow_rankings = []
rel_sets      = []

TOPK = 5

for qtext, label in QUERIES:
    rel = set(relevant_doc_indices(TEST_DOCS, label))
    rel_sets.append(rel)

    # get rankings
    lsa_order  = rank_with_lsa_map(qtext, lsa_model, LSA_TEST_EMBEDS)
    cbow_order = rank_with_cbow_map(qtext, cbow, CBOW_TEST_EMBEDS)

    lsa_rankings.append(lsa_order)
    cbow_rankings.append(cbow_order)

    # inspect per-query behavior
    print(f"[TASK3] Query='{qtext}' | (label={label})")

    print("   LSA  top-5 document labels:", [TEST_DOCS[i].label for i in lsa_order[:TOPK]])

    print("   CBOW top-5 document labels:", [TEST_DOCS[i].label for i in cbow_order[:TOPK]])
    print()

# Compute MAP
map_lsa  = mean_average_precision(lsa_rankings,  rel_sets)
map_cbow = mean_average_precision(cbow_rankings, rel_sets)

print(f"[TASK3 RESULT_CHECKING_POINT] MAP_LSA={map_lsa:.3f} | MAP_CBOW={map_cbow:.3f}")

In [ ]:
# ------------------------- TESTS & EXPECTED OUTPUT -------------------------
# [TASK3] Embedding dimensionality:
#   LSA  dim (k) = 10
#   CBOW dim (d) = 100
#
# [TASK3] Query='how do i fix problems on my mac ...' | (label=mac)
#    LSA top-5 document labels: ['autos', 'mac', ...]
#    CBOW top-5 document labels: ['mac', 'space', ...]
#
# ...
#
# [TASK3] Query='discussion about hockey teams ...'  | (label=hockey)
#    LSA top-5 document labels: ['hockey', 'space', ...]
#    CBOW top-5 document labels: ['hockey', 'autos', ...]
#
# [TASK3 RESULT_CHECKING_POINT] MAP_LSA=0.506 | MAP_CBOW=0.411
# ---------------------------------------------------------------------------

# BONUS (2 pts): Solve any task with an LLM

**Goal.**  
Pick **one** of the homework tasks (Task 1, Task 2, or Task 3) and solve it using an **LLM**. Provide the **LLM** with the **task description** and any **starter/prerequisite code** it depends on, ask it to generate a complete **code solution** first, then run that **generated code** here in the code notebook yourself. Finally, document what you did and **compare** the LLM’s result to your own pipeline.

**What to deliver below.**
1) **LLM used** (name + version, e.g., “Llama-3-8B-Instruct”, “GPT-x”, “Claude-x”, “Mistral-x”, etc.).  
2) **Prompt(s)** you used.  
3) **LLM output** — copy and paste the generated code.  
4) **Comparison** to your solution: what matches or differs (quantitative or qualitative).  
5) **Reflection**: what the LLM was **good at** vs **bad at**, what it got **right** vs **wrong**.

> **No code required.** You do **not** need to run, share, or submit any code used for the LLM generation. Provide only the deliverables listed above.
> You may use any LLMs through any interface (API, web UI, local inference).


# End of HW 02